In [1]:
import pygmt
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import os
import gsw

In [2]:
# Pasta onde os arquivos estão localizados
pasta = 'C:\\Users\\gabri\\Documents\\Pasta_Atual\\Pasta_CMIP6_TS_Volumetric\\Medias_nc\\25S_20N'

#Pasta para salvar os arquivos
saves = "C:\\Users\\gabri\\Documents\\Pasta_Atual\\Pasta_CMIP6_TS_Volumetric\\Volumetric_Plots_2\\Dataframes_25S_20N"

# Lista os arquivos na pasta
arquivos = os.listdir(pasta)

# Loop pelos arquivos
for arquivo in arquivos:
    if "historical" in arquivo:
        caminho_arquivo = os.path.join(pasta, arquivo)
        
        # Abre o arquivo usando xarray
        ds = xr.open_dataset(caminho_arquivo)
        
        #Transforma a Profundidade para pressão.
        ds['p'] = gsw.p_from_z(-ds['lev'], ds['lat'])

        #Transforma salinidade preformada para salinidade absoluta.
        ds['so'] = gsw.SA_from_Sstar(ds['so'], ds['p'], ds['lon'],ds['lat'])
        
        #Seleciona o intervalo de profundidade
        ds_lev = ds.sel(lev=slice(300, 1500))

        if 'lev_bnds' in ds_lev:
            ds_lev = ds_lev.drop('lev_bnds')
        
        if 'p' in ds_lev:
            ds_lev = ds_lev.drop('p')
        
        #Cria a variável volume, de acordo com a área para cada 5m
        ds_lev["vol"] = 5 * ds_lev["area"]

        #Cria a lista para os intervalos de temperatura 
        lista_temp = []           #Cria uma lista vazia
        i = -2                    #Define um minimo para a temperatura 
        lista_temp.append(i)      #Adiciona esse minimo a lista
        while round(i, 2) < 20:   #Define um máxio para a temperatura, o round é só para garantir que 19.999 seja 20.
            i += 0.2
            lista_temp.append(round(i, 2))   #round para garantir duas casas decimais e não ex: 2.400002.

        #Cria a lista para os intervalos de salinidade--> Mesmas considerações para a lista de temperatura
        lista_sal = []
        i=33
        lista_sal.append(i)
        while round(i, 3) < 37:
            i+=0.02
            lista_sal.append(round(i, 3))
            
        #Cria um dataframe para armazenar 
        df_final = pd.DataFrame()

        df = ds_lev.to_dataframe().dropna().reset_index()

        # Aplica o pd.cut com os limites definidos.
        cut_temp = pd.cut(df['thetao'], bins=lista_temp, include_lowest=True)   #Bins separa em 1-1.02; 1.02-1.04 ...
        cut_salin = pd.cut(df['so'], bins=lista_sal, include_lowest=True)       ##Bins separa em 1-1.002; 1.002-1.004 ...

        # Agrupa os dados pelos cruzamentos das categorias. 
        agrupado = df.groupby([cut_temp, cut_salin])['vol'].sum().reset_index()

        # Concatena meus dados em um dataframe final
        df_final = pd.concat([df_final, agrupado], ignore_index=True)

        # Cria as listas vazias
        thetao_medias = []
        so_medias = []

        # Calcula as médias dos intervalos e adicionar às listas
        for _, row in df_final.iterrows():
            thetao_medias.append(row['thetao'].mid)
            so_medias.append(row['so'].mid)

        # Adiciona as listas como novas colunas ao df_agrupado
        df_final['thetao_media'] = thetao_medias
        df_final['so_media'] = so_medias

        df_final = df_final[["thetao_media", "so_media", "vol"]]

        # Renomeia
        df_final.rename(columns={"thetao_media": "thetao", "so_media": "so"}, inplace=True)
        
        # Extrai o nome base do arquivo (sem o caminho)
        nome_base = os.path.basename(arquivo)

        # Extrai a parte relevante do nome para usar como nome do DataFrame
        nome_partes = nome_base.split('.')
        nome_dataframe = '.'.join(nome_partes[0:4])
        
        # Define o caminho de saída para o arquivo parquet com base no nome do DataFrame
        caminho_saida = os.path.join(saves, f"{nome_dataframe}.parquet")

        # Salva o DataFrame em formato parquet
        df_final.to_parquet(caminho_saida)
        
        print('Processamento Concluído: {}'.format(nome_dataframe))
        # Fecha o dataset xarray
        ds.close()

C:\Users\gabri\AppData\Local\Temp\ipykernel_28164\2214460626.py:28: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  ds_lev = ds_lev.drop('lev_bnds')
C:\Users\gabri\AppData\Local\Temp\ipykernel_28164\2214460626.py:31: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  ds_lev = ds_lev.drop('p')


Processamento Concluído: CMIP.CAMS.CAMS-CSM1-0.historical


C:\Users\gabri\AppData\Local\Temp\ipykernel_28164\2214460626.py:31: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  ds_lev = ds_lev.drop('p')


Processamento Concluído: CMIP.CNRM-CERFACS.CNRM-ESM2-1.historical


C:\Users\gabri\AppData\Local\Temp\ipykernel_28164\2214460626.py:28: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  ds_lev = ds_lev.drop('lev_bnds')
C:\Users\gabri\AppData\Local\Temp\ipykernel_28164\2214460626.py:31: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  ds_lev = ds_lev.drop('p')


...

Processamento Concluído: CMIP.MIROC.MIROC6.historical


C:\Users\gabri\AppData\Local\Temp\ipykernel_28164\2214460626.py:28: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  ds_lev = ds_lev.drop('lev_bnds')
C:\Users\gabri\AppData\Local\Temp\ipykernel_28164\2214460626.py:31: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  ds_lev = ds_lev.drop('p')


Processamento Concluído: CMIP.NCAR.CESM2.historical


C:\Users\gabri\AppData\Local\Temp\ipykernel_28164\2214460626.py:28: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  ds_lev = ds_lev.drop('lev_bnds')
C:\Users\gabri\AppData\Local\Temp\ipykernel_28164\2214460626.py:31: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  ds_lev = ds_lev.drop('p')


Processamento Concluído: CMIP.NOAA-GFDL.GFDL-ESM4.historical


## Criação dataframe SSP585

In [3]:
# Pasta onde os arquivos estão localizados
pasta = 'C:\\Users\\gabri\\Documents\\Pasta_Atual\\Pasta_CMIP6_TS_Volumetric\\Medias_nc\\25S_20N'

#Pasta para salvar os arquivos
saves = "C:\\Users\\gabri\\Documents\\Pasta_Atual\\Pasta_CMIP6_TS_Volumetric\\Volumetric_Plots_2\\Dataframes_25S_20N"

# Lista os arquivos na pasta
arquivos = os.listdir(pasta)

# Loop pelos arquivos
for arquivo in arquivos:
    if "ssp585" in arquivo:
        caminho_arquivo = os.path.join(pasta, arquivo)
        
        # Abre o arquivo usando xarray
        ds = xr.open_dataset(caminho_arquivo)
        
        #Transforma a Profundidade para pressão.
        ds['p'] = gsw.p_from_z(-ds['lev'], ds['lat'])

        #Transforma salinidade preformada para salinidade absoluta.
        ds['so'] = gsw.SA_from_Sstar(ds['so'], ds['p'], ds['lon'],ds['lat'])
        
        #Seleciona o intervalo de 500 a 1500m
        ds_lev = ds.sel(lev=slice(300, 1500))
        
        #Cria a variável volume, de acordo com a área para cada 5m
        ds_lev["vol"] = 5 * ds_lev["area"]
        
        if 'lev_bnds' in ds_lev:
            ds_lev = ds_lev.drop('lev_bnds')
        
        if 'p' in ds_lev:
            ds_lev = ds_lev.drop('p')
       
        #Cria a lista para os intervalos de temperatura 
        lista_temp = []
        i = -2
        lista_temp.append(i)
        while round(i, 2) < 20:
            i += 0.2
            lista_temp.append(round(i, 2))

        #Cria a lista para os intervalos de salinidade
        lista_sal = []
        i=33
        lista_sal.append(i)
        while round(i, 3) < 37:
            i+=0.02
            lista_sal.append(round(i, 3))
            
        #Cria um dataframe para armazenar 
        df_final = pd.DataFrame()

        df = ds_lev.to_dataframe().dropna().reset_index()

        # Aplica o pd.cut com os limites definidos
        cut_temp = pd.cut(df['thetao'], bins=lista_temp, include_lowest=True)
        cut_salin = pd.cut(df['so'], bins=lista_sal, include_lowest=True)

        # Agrupa os dados pelos cruzamentos das categorias
        agrupado = df.groupby([cut_temp, cut_salin])['vol'].sum().reset_index()

        # Concatena meus dados em um dataframe final
        df_final = pd.concat([df_final, agrupado], ignore_index=True)

        # Cria as listas vazias
        thetao_medias = []
        so_medias = []

        # Calcula as médias dos intervalos e adicionar às listas
        for _, row in df_final.iterrows():
            thetao_medias.append(row['thetao'].mid)
            so_medias.append(row['so'].mid)

        # Adiciona as listas como novas colunas ao df_agrupado
        df_final['thetao_media'] = thetao_medias
        df_final['so_media'] = so_medias

        df_final = df_final[["thetao_media", "so_media", "vol"]]

        # Renomeia
        df_final.rename(columns={"thetao_media": "thetao", "so_media": "so"}, inplace=True)
        
        # Extrai o nome base do arquivo (sem o caminho)
        nome_base = os.path.basename(arquivo)

        # Extrai a parte relevante do nome para usar como nome do DataFrame
        nome_partes = nome_base.split('.')
        nome_dataframe = '.'.join(nome_partes[0:4])
        
        # Define o caminho de saída para o arquivo parquet com base no nome do DataFrame
        caminho_saida = os.path.join(saves, f"{nome_dataframe}.parquet")

        # Salva o DataFrame em formato parquet
        df_final.to_parquet(caminho_saida)
        
        print('Processamento Concluído: {}'.format(nome_dataframe))
        # Fecha o dataset xarray
        ds.close()

C:\Users\gabri\AppData\Local\Temp\ipykernel_17976\2397048005.py:31: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  ds_lev = ds_lev.drop('lev_bnds')
C:\Users\gabri\AppData\Local\Temp\ipykernel_17976\2397048005.py:34: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  ds_lev = ds_lev.drop('p')


Processamento Concluído: ScenarioMIP.CAMS.CAMS-CSM1-0.ssp585


C:\Users\gabri\AppData\Local\Temp\ipykernel_17976\2397048005.py:34: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  ds_lev = ds_lev.drop('p')


Processamento Concluído: ScenarioMIP.CNRM-CERFACS.CNRM-ESM2-1.ssp585


C:\Users\gabri\AppData\Local\Temp\ipykernel_17976\2397048005.py:31: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  ds_lev = ds_lev.drop('lev_bnds')
C:\Users\gabri\AppData\Local\Temp\ipykernel_17976\2397048005.py:34: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  ds_lev = ds_lev.drop('p')


...

Processamento Concluído: ScenarioMIP.MIROC.MIROC6.ssp585


C:\Users\gabri\AppData\Local\Temp\ipykernel_17976\2397048005.py:31: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  ds_lev = ds_lev.drop('lev_bnds')
C:\Users\gabri\AppData\Local\Temp\ipykernel_17976\2397048005.py:34: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  ds_lev = ds_lev.drop('p')


Processamento Concluído: ScenarioMIP.NCAR.CESM2.ssp585


C:\Users\gabri\AppData\Local\Temp\ipykernel_17976\2397048005.py:31: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  ds_lev = ds_lev.drop('lev_bnds')
C:\Users\gabri\AppData\Local\Temp\ipykernel_17976\2397048005.py:34: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  ds_lev = ds_lev.drop('p')


Processamento Concluído: ScenarioMIP.NOAA-GFDL.GFDL-ESM4.ssp585
